In [24]:
import pandas as pd
import numpy as np

import matplotlib as plt
import seaborn as scp

import cv2

import random

##### Lectura y limpieza de los datos

In [37]:
product = pd.read_csv("./dataset/product_data.csv")
outfits = pd.read_csv("./dataset/outfit_data.csv")
product

,cod_modelo_color,cod_color_code,des_color_specification_esp,des_agrup_color_eng,des_sex,des_age,des_line,des_fabric,des_product_category,des_product_aggregated_family,des_product_family,des_product_type,des_filename
0,41085800-02,02,OFFWHITE,WHITE,Female,Adult,SHE,P-PLANA,Bottoms,Trousers & leggings,Trousers,Trousers,datathon/images/2019_41085800_02.jpg
1,53000586-TO,TO,TEJANO OSCURO,BLUE,Female,Adult,SHE,J-JEANS,Bottoms,Jeans,Jeans,Jeans,datathon/images/2019_53000586_TO.jpg
2,53030601-81,81,ROSA PASTEL,PINK,Female,Adult,SHE,P-PLANA,"Dresses, jumpsuits and Complete set",Dresses and jumpsuits,Dresses,Dress,datathon/images/2019_53030601_81.jpg
3,53050730-15,15,MOSTAZA,YELLOW,Female,Adult,SHE,P-PLANA,"Dresses, jumpsuits and Complete set",Dresses and jumpsuits,Dresses,Dress,datathon/images/2019_53050730_15.jpg
4,53070773-70,70,ROJO,RED,Female,Adult,SHE,P-PLANA,Tops,Shirts,Shirt,Shirt,datathon/images/2019_53070773_70.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9217,57066027-99,99,NEGRO,GREY,Female,Adult,SHE,P-PLANA,"Dresses, jumpsuits and Complete set",Dresses and jumpsuits,Dresses,Dress,datathon/images/2023_57066027_99.jpg
9218,57076028-92,92,GRIS,GREY,Female,Adult,SHE,P-PLANA,Bottoms,Skirts and shorts,Shorts,Shorts,datathon/images/2023_57076028_92.jpg
9219,57068275-05,05,CRUDO,WHITE,Female,Adult,SHE,C-COMPLEMENTOS,"Accesories, Swim and Intimate",Swim and intimate,Intimate,Socks,datathon/images/2023_57068275_05.jpg
9220,57067125-78,78,BURDEOS,RED,Female,Adult,SHE,K-CIRCULAR,"Dresses, jumpsuits and Complete set",Dresses and jumpsuits,Dresses,Dress,datathon/images/2023_57067125_78.jpg


In [26]:
#Deleting all the products that are not clothes
product = product[(product.des_product_category!= 'Beauty') & (product.des_product_category!= 'Home')]

# Delete all products for men and kids due to desbalance
product = product[(product.des_line== 'SHE')]
product = product[(product.des_sex== 'Female')]
product = product[(product.des_age== 'Adult')]

product =product.drop(['des_sex','des_line','des_age'], axis = 1)

# Delete the deleted products from outfits
outfits = outfits[outfits.cod_modelo_color.isin(product.cod_modelo_color.unique())]

In [27]:
product.des_product_category.unique()

array(['Bottoms', 'Dresses, jumpsuits and Complete set', 'Tops',
       'Accesories, Swim and Intimate', 'Outerwear'], dtype=object)

In [28]:
def getProductIdx(idx):
    print("DATOS DEL PRODUCTO")
    print(product.iloc[idx])
    print('OUTFITS APPEARING IN:')
    code = product.iloc[idx].cod_modelo_color
    print(list(outfits[outfits.cod_modelo_color==code]['cod_outfit'].unique()))
    print('IMAGE')
    image_path =product.loc[idx]['des_filename']
    img = cv2.imread(image_path)
    plt.imshow(img)

---
##### Métodos necesarios

In [29]:
def getProductByCode(code):
    print("DATOS DEL PRODUCTO")
    print(product.loc[product.cod_modelo_color == code])
    print('OUTFITS APPEARING IN:')
    print(list(outfits[outfits.cod_modelo_color==code]['cod_outfit'].unique()))
    print('IMAGE')
    image_path =product[product.cod_modelo_color == code]['des_filename'].values[0]
    img = cv2.imread(image_path)
    plt.imshow(img)

---
##### Definición del estado

In [36]:
class state:
    def __init__(self, top=None, bot=None, shoes=None, _h = None):
        self._top = top
        self._bot = bot
        self._shoes = shoes
        self._h = _h
        
        if (_top!= None or _shoes!=None or _bot!=None): _h = computeHeuristic()
        
    def __repr__(self, obj):
        self
    
    def get_top(self):
        return self._top
    def get_bot(self):
        return self._bot
    def get_shoes(self):
        return self._zapatos
    
    def set_top_part(self, top):
        self._top = top
    def set_bottom_part(self, bot):
        self._bot = bot
    def set_shoes(self, shoes):
        self._shoes = shoes
    
    def set_heuristica(self, h):
        self._h = h
    def get_heusistica(self):
        return self.old_score

In [35]:
def computeHeuristic():
    return 1

In [39]:
def getNewProductSuccessor(estado, part):    
    match part:
        case "top":
            data  = product.loc[product['des_product_category'] == "Top"]
        case "bot":
            data = product.loc[product['des_product_category'] == "Bottom"]
        case "shoes":
            data  = product.loc[product['des_product_family'] == "Footwear"]
    succ = None
    while(succ == None):
        randomObs = random.sample(data, 1)
        newProd = randomObs.cod_modelo_color
        newStateH = computeHeuristic(newState)
        return newProd if newStateH > estado.get_heuristica() else None

In [34]:
def generateSuccessors(estado):
    successors = []
    random_seed = None
    
    # top sucesores
    topPart = getNewProdSucc(estado, "top")
    if(topPart != None):
        newState = State(topPart, estado.get_bot(), estado.get_shoes())
        successors.append(newState, newState.get_heuristica())
    
    # bottom sucesores
    botPart = getNewProdSucc(estado, "bot")
    if(bottomPart != None):
        newState = State(estado.get_top(), botPart, estado.get_shoes())
        successors.append(newState, newState.get_heuristica())
        
    # zapatos sucesores
    shoesPart = getNewProdSucc(estado, "shoes")
    if(bottomPart != None):
        newState = State(estado.get_top(), estado.get_bot(), shoesPart)
        successors.append(newState, newState.get_heuristica())
    
    if (len(successors) == 0): return estado
    maxH = successors[0].item(0)
    bestSuccessor = succ.item(0)
    for succ in successors:
        if(maxH > succ.item(1)):
            maxH = succ.item(1)
            bestSuccessor = succ.item(0)
    
    return generateSuccessors(bestSuccessor)